In [1]:
# Mount path for Google Colab
from google.colab import drive
drive.mount('/content/drive/')

root_path = '/content/drive/My Drive/comp90042-project1/project-data/'

Mounted at /content/drive/


In [ ]:
# Inspect train data using pandas
import pandas as pd

train_df = pd.read_json(root_path + 'train.data.jsonl', lines=True)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345
0,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated

In [29]:
# Build a preprocessing function for vectorizing words
import json
import pandas as pd

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

class PreprocessDataset():

    def __init__(self, datafile, labelfile, devfile, devlabel, testfile, num_replies):
        # Define constants
        self.REPLY_ID = 'in_reply_to_status_id_str'
        self.TEXT = 'text'

        # The specified number of replies to be considered
        self.num_replies = num_replies

        # Initialise label encoder
        self.le = preprocessing.LabelEncoder()

        # Store train dataset in pandas dataframe
        self.train_df = pd.read_json(datafile, lines=True)
        self.dev_df = pd.read_json(devfile, lines=True)
        self.test_df = pd.read_json(testfile, lines=True)

        # Store labels as a list encoded with 0 and 1
        # If labels == None, then preprocessing is done for test dataset
        if labelfile != None:
            with open(labelfile, 'r') as f:
                load_labels = json.load(f)
                self.labels = self.le.fit_transform(list(load_labels.values())).tolist()
        else:
            self.labels = None
        
        if devlabel != None:
            with open(devlabel, 'r') as f:
                load_labels = json.load(f)
                self.dev_labels = self.le.fit_transform(list(load_labels.values())).tolist()
        else:
            self.dev_labels = None
        
        # Initialise vectorizer and TfIdf Transformer
        self.vectorizer = CountVectorizer(stop_words='english')
        self.tf_idf_transformer = TfidfTransformer()
    
    def get_text_list(self, dataframe):
        # Get the list of texts based on the num_replies
        text_list = []
        for index, row_tweet in dataframe.iterrows():
            counter = 0
            tweet_text = ''
            for tweet in row_tweet:
                if tweet == None or counter == self.num_replies + 1:
                    break
                tweet_text += tweet[self.TEXT]
                counter += 1
            text_list.append(tweet_text)
        
        return text_list
        

    def get_train_data(self):
        # Preprocess using vectorizer and TfIdf Transformer
        text_vector = self.vectorizer.fit_transform(self.get_text_list(self.train_df))
        text_transformed = self.tf_idf_transformer.fit_transform(text_vector)
        
        return text_transformed
    
    def get_dev_data(self):
        # Preprocess using vectorizer and TfIdf Transformer
        dev_text_vector = self.vectorizer.transform(self.get_text_list(self.dev_df))
        dev_text_transformed = self.tf_idf_transformer.transform(dev_text_vector)
        
        return dev_text_transformed
    
    def get_test_data(self):
        # Preprocess using vectorizer and TfIdf Transformer
        test_text_vector = self.vectorizer.transform(self.get_text_list(self.test_df))
        test_text_transformed = self.tf_idf_transformer.transform(test_text_vector)
        
        return test_text_transformed


In [30]:
# Load the data using PreprocessDataset

data_X = PreprocessDataset(datafile=root_path + 'train.data.jsonl',
                           labelfile=root_path + 'train.label.json',
                           devfile=root_path + 'dev.data.jsonl',
                           devlabel=root_path + 'dev.label.json',
                           testfile=root_path + 'test.data.jsonl',
                           num_replies=20)


print('Preprocess is finished!')

Preprocess is finished!


In [31]:
# Train the data using logistic regression
from sklearn.linear_model import LogisticRegression

logit_cls = LogisticRegression().fit(data_X.get_train_data(), data_X.labels)

In [32]:
# Check the score on the dev set
import numpy as np

dev_preds = logit_cls.predict(data_X.get_dev_data())
np.mean(dev_preds == data_X.dev_labels)

0.8275862068965517

In [40]:
# Predict the test set using logistic regression
REPLY_ID = REPLY_ID = 'in_reply_to_status_id_str'
ID = 'id_str'

id_list = []
predictions = dict()

test_preds = logit_cls.predict(data_X.get_test_data())

for index, row in data_X.test_df.iterrows():
    for tweet in row:
        if tweet[REPLY_ID] == None:
            id_list.append(tweet[ID])
            break

for id, pred in zip(id_list, test_preds):
    if pred == 0:
        predictions[id] = 'non-rumour'
    else:
        predictions[id] = 'rumour'

with open(root_path + 'test-output.json', 'w') as outfile:
    json.dump(predictions, outfile)

print('Predictions generated!!')

Predictions generated!!


In [42]:
# Train the data using multi-layer perceptron
from sklearn.neural_network import MLPClassifier

mlp_cls = MLPClassifier().fit(data_X.get_train_data(), data_X.labels)

In [43]:
# Check the score on the dev set
import numpy as np

dev_preds = mlp_cls.predict(data_X.get_dev_data())
np.mean(dev_preds == data_X.dev_labels)

0.843103448275862

In [44]:
# Predict the test set using multi-layer perceptron
REPLY_ID = REPLY_ID = 'in_reply_to_status_id_str'
ID = 'id_str'

id_list = []
predictions = dict()

test_preds = mlp_cls.predict(data_X.get_test_data())

for index, row in data_X.test_df.iterrows():
    for tweet in row:
        if tweet[REPLY_ID] == None:
            id_list.append(tweet[ID])
            break

for id, pred in zip(id_list, test_preds):
    if pred == 0:
        predictions[id] = 'non-rumour'
    else:
        predictions[id] = 'rumour'

with open(root_path + 'test-output.json', 'w') as outfile:
    json.dump(predictions, outfile)

print('Predictions generated!!')

Predictions generated!!
